# Phase 3 — Définition du défaut (Target)

## Objectif
Créer une variable cible binaire `default_flag` à partir de la colonne `loan_status`.

- default_flag = 1 → prêt en défaut
- default_flag = 0 → prêt non en défaut

La définition du défaut repose sur une règle métier explicite et documentée.


In [1]:
import pandas as pd

df = pd.read_csv("/Users/master/Downloads/credit-climate-risk-lab/data/raw/lendingclub.csv", low_memory=False)

In [4]:
status_counts = df.loan_status.value_counts().reset_index()
status_counts.columns = ["loan_status", "count"]
status_counts 

,loan_status,count
0,Fully Paid,1076751
1,Current,878317
2,Charged Off,268559
3,Late (31-120 days),21467
4,In Grace Period,8436
5,Late (16-30 days),4349
6,Does not meet the credit policy. Status:Fully ...,1988
7,Does not meet the credit policy. Status:Charge...,761
8,Default,40


### Les types de satus de prêts utilisables pour notres modèles.
___

- Fully Paid : default_flag = 0 (La réponse à la question : Ce prêts fait il est (False : 0))
- Charged Off = default_flag = 1 (Charged off a un fort % de faire défault)
- default = 1 

In [7]:
valid_status = ["Fully Paid", "Charged Off", "Default"]
df_model = df[df["loan_status"].isin(valid_status)].copy()
df_model.loan_status.value_counts()

loan_status
Fully Paid     1076751
Charged Off     268559
Default             40
Name: count, dtype: int64

In [15]:
# Création de la variable cible binaire
df_model["default_flag"] = df_model.loan_status.apply(lambda x: 1 if x in ["Charged Off", "Default"] else 0)

df_model[["loan_status", "default_flag"]].head(50)

,loan_status,default_flag
0,Fully Paid,0
1,Fully Paid,0
2,Fully Paid,0
4,Fully Paid,0
5,Fully Paid,0
6,Fully Paid,0
7,Fully Paid,0
8,Fully Paid,0
9,Fully Paid,0
12,Fully Paid,0


In [16]:
# Taux de défaut

df_model.default_flag.value_counts(normalize = True)

default_flag
0    0.80035
1    0.19965
Name: proportion, dtype: float64

### Vérification du taux de défaut

La distribution de la variable `default_flag` montre qu’environ 20 % des prêts
terminés sont en défaut, contre 80 % non défaillants.

Ce niveau de déséquilibre est modéré et cohérent avec un portefeuille de crédit
à la consommation non garanti. Il permet l’utilisation de modèles standards
comme la régression logistique.


In [18]:
target_summary = (
    df_model[["loan_status", "default_flag"]]
    .value_counts()
    .reset_index(name="count")
)

target_summary.to_csv(
   "/Users/master/Downloads/credit-climate-risk-lab/reports/tableau_exports/03_default_definition_summary.csv",
    index=False
)

target_summary


,loan_status,default_flag,count
0,Fully Paid,0,1076751
1,Charged Off,1,268559
2,Default,1,40


### Justification de la définition du défaut

La variable `default_flag` est définie à partir de la colonne `loan_status`.
Les statuts "Charged Off" et "Default" sont considérés comme des défauts.
Les prêts "Fully Paid" sont considérés comme non défaillants.

Les prêts encore en cours ("Current", "Late") sont exclus afin d’éviter toute ambiguïté,
conformément aux pratiques de modélisation du risque de crédit.
